In [1]:
from utils import *
import numpy as np
from numpy.random import seed
import tensorflow as tf
import datetime
import pandas as pd 

# Custom imports
from nets import unet, unet_2d, unet_bi, unetpp, unetpp_2d, scSEunet, scSEunetpp
from config import get_config

cfg = get_config()

X,  Y, paths  = load_dataset('data_full/test/', return_paths=True, pad=28)
Xx, Y, paths  = load_dataset('data_full/test/', return_paths=True, addcoords=True, pad=28)

Using TensorFlow backend.
100%|██████████| 10/10 [00:01<00:00,  6.90it/s]


In [2]:
model = scSEunet(256, 256, 64, 1)
model.load_weights('models/256_model_scseunet2_fl.p5')

In [3]:
outs = model.predict(X, batch_size=1)

In [25]:
if type(outs) == type([]):
    outs = outs[-1]

export_outs(X, Y, outs, 'outs/global/', paths=paths)

def summary(outs, Y):
    tabledata = []
    means = [0] * 5
    for i in range(outs.shape[0]):
        row = [paths[i]]
        mean = 0
        for j in range(1, 6):
            yt = Y[i,:,:,:,j]
            yp = outs[i,:,:,:,j]

            dice = dice_coef(yt, yp, numpy=True) * 100
            row.append('%.2f' % dice)
            
            mean += (dice / 5)
            means[j-1] += (dice / outs.shape[0])

        row.append('%.2f' % mean)
        tabledata.append(row)
    tabledata.append(['Mean'] + ['%.2f' % m for m in means] + ['%.2f' % np.mean(means)])
    df = pd.DataFrame(tabledata, columns=['Name', 'Liver', 'Kidney', 'Stomach', 'Duodenum', 'Largebowel', 'Mean'])
    df = df.set_index('Name')
    return df

Exporting...


In [26]:
summary(outs, Y)

,Liver,Kidney,Stomach,Duodenum,Largebowel,Mean
Name,,,,,,
orig_data_111.mat,93.08,90.82,86.73,20.40,64.00,71.01
orig_data_112.mat,93.92,83.42,89.67,15.35,68.81,70.24
orig_data_113.mat,93.50,4.90,77.98,12.24,47.54,47.23
orig_data_114.mat,93.13,90.81,82.06,26.71,87.15,75.97
orig_data_115.mat,95.70,90.91,83.51,47.71,79.65,79.50
orig_data_116.mat,94.23,71.50,88.46,39.86,84.52,75.71
orig_data_117.mat,95.92,94.63,68.51,30.54,85.40,75.00
orig_data_118.mat,95.15,94.92,80.89,43.50,81.17,79.13
orig_data_119.mat,94.64,93.70,86.09,67.43,82.50,84.87


# Local refinement

In [27]:
X = np.concatenate((Xx, outs), axis=4)
X.shape

(10, 256, 256, 64, 10)

In [28]:
output = np.zeros(X.shape[:-1] + (6,))
output.shape

(10, 256, 256, 64, 6)

In [29]:
patch_model = unet(64, 64, 16, 10, bn=True)
patch_model.load_weights('models/model_1.unet_patches.p5')

In [31]:
steps = (32,32,8)

In [32]:
for i in range(0,256-64+1,steps[1]):
    for j in range(0,256-64+1,steps[0]):
        for k in range(0,64-16+1,steps[2]):
            iend = i+64
            jend = j+64
            kend = k+16
            patch = X[:,j:jend,i:iend,k:kend]
            out = patch_model(patch)
            output[:,j:jend,i:iend,k:kend] += out


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyboardInterrupt: 

In [ ]:
Y.shape

In [ ]:
seg_local = output.argmax(axis=-1)
seg_global = outs.argmax(axis=-1)
seg_actual = Y.argmax(axis=-1)
slc = 40

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.imshow(seg_local[0,:,:,slc])
plt.subplot(132)
plt.imshow(seg_global[0,:,:,slc])
plt.subplot(133)
plt.imshow(seg_actual[0,:,:,slc])

In [ ]:
if type(output) == type([]):
    output = output[-1]

export_outs(X, Y, output, 'outs/local/', paths=paths)

summary(Y, output)